##### - DNN기반 회귀 구현
 - 데이터셋 : IRIS.csv
 - 피쳐/속성 : 3개 Setal_Length, Sepal_Width, Petal_Width
 - 타겟/라벨 : 1개 Petal_Length
 - 학습-방법 : 지도학습 > 회귀
 - 알고 리즘 : 인공신경망(ANN) -> 심층신경망(MLP), DNN : 은닉층이 많은 구성
 - 프레임워크 : Pytorch
 

In [16]:
# 모듈 로딩
# - Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.classification import F1Score, BinaryF1Score
from torchmetrics.classification import BinaryConfusionMatrix
from torchinfo import summary
 
# Data 및 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [17]:
### 데이터로딩
DATA_FILE = 'iris.csv'

### CSV => DataFrame
irisDF = pd.read_csv(DATA_FILE)

### 확인
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [18]:
### 타겟 변경 => 정수화, 클래스 3개 => 2개
irisDF['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [19]:
# 클래스 3개 => 2개
irisDF['variety'] = (irisDF['variety'] == 'Setosa')
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,True


In [20]:
irisDF['variety'] = irisDF['variety'].astype('int')
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,1


In [21]:
irisDF['variety'].unique()

array([1, 0])

In [22]:
### 타겟 정수화
labels = dict(zip(irisDF['variety'].unique().tolist(), range(3)))
print(f'labels => {labels}')


labels => {1: 0, 0: 1}


[2] 모델 클래스 설계 및 정의 <hr>
 - 클래스목적 : iris 데이터를 학습 및 추론 목적 
 - 클래스이름 : IrisRegModel
 - 부모클래스 : nn.Module
 - 매개__변수 : 층별 입출력 개수 고정하기 때문에 필요없음!
 - 속성__필드 : featureDF, targetDF, n_rows, n_features 
 - 기능  역할 : __init__()  : 모델 구조 설정, forward() : 순방향 학습, <= 오버라이딩(overriding)
 - 클래스구조 
    * 입력층 : 입력 4개 (피쳐)   출력 10개 (퍼셉트론 / 뉴런 10개 존재)
    * 은닉층 : 입력 10개         출력 30개 (퍼셉트론 / 뉴런 30개 존재)
    * 출력층 : 입력 5개         출력 1개 (2진분류)


 - 활성화함수
   * 클래스 형태 ==> nn.MESLoss, nn.ReLU ==> __init__() 메서드
   * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드

In [23]:
class IrisRegModel(nn.Module):
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(4,10)
        self.hidden_layer = nn.Linear(10,5)
        self.out_layer = nn.Linear(5,1)


    def forward(self, input_data):
        y = self.in_layer(input_data)   # f11w11 + f12w12 + f13w13 ... + f103w103 + b
        y = F.relu(y)                       

        # - 은닉층 : 10개의 숫자 값 (>= 0)
        y = self.hidden_layer(y)                 
        y = F.relu(y)                       

        # - 출력층 : 30개의 숫자 값 (>=0)
        return self.out_layer(y)                   # f31w31 .... f330w330 + b   // 3번의 30번

In [24]:
### 모델 인스턴스 생성
model = IrisRegModel()
print(model)

IrisRegModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=1, bias=True)
)


In [25]:
summary(model, input_size=(17,4))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [17, 1]                   --
├─Linear: 1-1                            [17, 10]                  50
├─Linear: 1-2                            [17, 5]                   55
├─Linear: 1-3                            [17, 1]                   6
Total params: 111
Trainable params: 111
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [26]:
class IrisDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]
        
    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화

        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values) # 그대로 가져오면 시리즈니까 values로 형 변환
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)
        # 피쳐와 타겟 변환

        return featureTS, targetTS

In [27]:
## 데이터셋 인스턴스 생성
# - DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]] # 2D (150,1)
targetDF = irisDF[irisDF.columns[-1:]]  # 2D (150,1)

# 커스텀데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF, targetDF)

# 데이터 로더 인스턴스 생성
irisDL = DataLoader(irisDS)
for feature, label in irisDL:
    print(feature.shape, label.shape, feature, label)
    break

torch.Size([1, 4]) torch.Size([1, 1]) tensor([[5.1000, 3.5000, 1.4000, 0.2000]]) tensor([[1.]])


[4] 학습 준비
 - 학습 횟수 : EPOCH            <- 처음 ~ 끝까지 공부하는 단위
 - 배치 크기 : BATCH_SIZE       <- 한번에 학습할 데이터셋 양
 - 위치 지정 : DEVICE           <- 텐서 저장 및 실행 위치 (GPU / CPU)
 - 학_습_률 : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001 ~ 0.1

In [ ]:
### 학습 진행 관련 설정
EPOCH = 1
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

 - 인스턴스 / 객체 : 모델, 데이터셋, 최적화 (, 손실함수 , 성능지표)

In [ ]:
# 모델 인스턴스
model = IrisBCFModel()

In [28]:
# DS과 DL 인스턴스
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)

trainDS = IrisDataset(X_train,y_train)
valDS = IrisDataset(X_val, y_val)
testDS = IrisDataset(X_test,y_test)

trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE)


NameError: name 'BATCH_SIZE' is not defined

In [ ]:
optimizer = optim.adam